In [41]:
import pandas as pd
import numpy as np
import re


In [42]:
#AISC Section Shapes 
shapes_frame = pd.read_excel("AISC Shapes Database v15.0.xlsx", na_filter=False, sheet_name=1)

In [62]:
def df_sect(x):
    '''Enter Column/Beam Size Group as a string i.e. "W14" or "HSS"'''
    sz_type = re.findall("[A-Z]",x)[0]
    df_col = shapes_frame.loc[(shapes_frame['Type'] == sz_type) & (shapes_frame['AISC_Manual_Label'].str.contains(x))]
    return df_col.sort_values(by=['Zx'],ascending=False).reset_index()

In [134]:
# find the deflection for a simple single bay moment frame
# Predefined to 15' wide by 20' tall
# 75 kip wind load at the top
# x 
def edk_risa_np(x):
    '''find the deflection for a simple single bay moment frame
        x is a column and beam couple which is entered as a list -> [column index, beam index]
        Predefined to:
        - 15' wide by 20' tall
        - 50 kip wind load at the top
        '''
    col_shape = col_sections.iloc[x[0]]['AISC_Manual_Label']
    bm_shape = bm_sections.iloc[x[1]]['AISC_Manual_Label']
    # Units used for the model in this example are inches and kips

    from PyNite import FEModel3D

    # Create a new finite element model
    MomentFrame = FEModel3D()

    w = 15*12
    ht = 20*12

    # Add nodes (frame is 15 ft wide x 12 ft tall)
    MomentFrame.add_node('N1', 0, 0, 0)
    MomentFrame.add_node('N2', 0, ht, 0)
    MomentFrame.add_node('N3', w, 0, 0)
    MomentFrame.add_node('N4', w, ht, 0)

    # Define column properties (use W10x33 from the AISC Manual):
    E = 29000*.80 # ksi
    G = 11200 # ksi
    col_Ix = shapes_frame.loc[shapes_frame['AISC_Manual_Label'].eq(col_shape)]['Ix'].item()
    col_Iy = shapes_frame.loc[shapes_frame['AISC_Manual_Label'].eq(col_shape)]['Iy'].item()
    col_J = shapes_frame.loc[shapes_frame['AISC_Manual_Label'].eq(col_shape)]['J'].item()
    col_A = shapes_frame.loc[shapes_frame['AISC_Manual_Label'].eq(col_shape)]['A'].item()
    col_wt = float(col_shape.split('X')[1])*-1/12000
    #print(col_Ix,col_Iy,col_J,col_A)

    # Define the columns
    MomentFrame.add_member('Col1', 'N1', 'N2', E, G, col_Iy, col_Ix, col_J, col_A)
    MomentFrame.add_member('Col2', 'N3', 'N4', E, G, col_Iy, col_Ix, col_J, col_A)

    # Define beam properties (Use W8x24)
    bm_Ix = shapes_frame.loc[shapes_frame['AISC_Manual_Label'].eq(bm_shape)]['Ix'].item()
    bm_Iy = shapes_frame.loc[shapes_frame['AISC_Manual_Label'].eq(bm_shape)]['Iy'].item()
    bm_J = shapes_frame.loc[shapes_frame['AISC_Manual_Label'].eq(bm_shape)]['J'].item()
    bm_A = shapes_frame.loc[shapes_frame['AISC_Manual_Label'].eq(bm_shape)]['A'].item()
    bm_wt = float(col_shape.split('X')[1])*-1/12000

    # Define the beams
    MomentFrame.add_member('Beam1', 'N2', 'N4', E, G, bm_Iy, bm_Ix, bm_J, bm_A)


    # Provide pinned supports at the bases of the columns
    MomentFrame.def_support('N1', support_DX=True, support_DY=True, support_DZ=True, support_RX=False, support_RY=False, support_RZ=False)
    MomentFrame.def_support('N3', support_DX=True, support_DY=True, support_DZ=True, support_RX=False, support_RY=False, support_RZ=False)


    # Add self weight dead loads to the frame
    # Note that we could leave 'x1' and 'x2' undefined below and it would default to the full member length
    # Note also that the direction uses lowercase notations to indicate member local coordinate systems
    MomentFrame.add_member_dist_load('Beam1', Direction='Fy', w1=bm_wt, w2=bm_wt, x1=0, x2=w, case='D')
    MomentFrame.add_member_pt_load('Beam1','Fy',-10,w/2)
    MomentFrame.def_releases('Beam1', Dxi=False, Dyi=False, Dzi=False, Rxi=False, Ryi=False, Rzi=False, Dxj=False, Dyj=False, Dzj=False, Rxj=False, Ryj=False, Rzj=False)
    
    MomentFrame.add_member_dist_load('Col1', Direction='Fx', w1=col_wt, w2=col_wt, x1=0, x2=ht, case='D')
    MomentFrame.add_member_dist_load('Col2', Direction='Fx', w1=col_wt, w2=col_wt, x1=0, x2=ht, case='D')

    # Add a nodal wind load of 10 kips at the left side of the frame
    # Note that the direction uses uppercase notation to indicate model global coordinate system
    MomentFrame.add_node_load('N2', Direction='FX', P=50, case='W')

    # Create two load combinations
    MomentFrame.add_load_combo('1.0D+1.0W', factors={'D':1.0, 'W':1.0})

    # Perform a P-Big Delta analysis on the frame
    # Note that to capture P-little delta effects the members should idealy be broken into three segments each
    MomentFrame.analyze_PDelta()

    # A first-order analysis could be done with the following line instead
    #MomentFrame.Analyze()

    ## Display the deformed shape of the structure magnified 50 times with the text height 5 model units (inches) high
    #from PyNite import Visualization
    #Visualization.RenderModel(MomentFrame, text_height=5, deformed_shape=True, deformed_scale=50, combo_name='1.0D+1.0W')

    defl = MomentFrame.Nodes['N2'].DX['1.0D+1.0W']
    
    wt_col = shapes_frame[shapes_frame['AISC_Manual_Label']==col_shape].W.item()
    wt_bm = shapes_frame[shapes_frame['AISC_Manual_Label']==bm_shape].W.item()
    tot_wt = 2*wt_col*ht/12 + wt_bm*w/12
    
    return defl, tot_wt

def scwb(x):
    '''Strong Column Weak Beam constraint
    where beam capacity must be >1.2 times the column capacity
    (this is an approximation)
    '''
    return -col_sections['Zx'][x[0]]+1.2*bm_sections['Zx'][x[1]]

In [128]:
import numpy as np
from pymoo.factory import get_algorithm, get_crossover, get_mutation, get_sampling
from pymoo.optimize import minimize
from pymoo.core.problem import Problem

col_sections = df_sect('W21')
bm_sections = df_sect('W24')

up_bound_col=len(col_sections)-1
up_bound_bm=len(bm_sections)-1



class MyProblem(Problem):
    def __init__(self):
        super().__init__(n_var=2, n_obj=2, n_constr=3,
                         xl=np.array([0,0]), xu=np.array([up_bound_col, up_bound_bm]),type_var=int)
    
    def _evaluate(self, x, out, *args, **kwargs):
        obj = np.apply_along_axis(edk_risa_np, axis=1, arr=x)
        f1 = obj[:,0]
        f2 = obj[:,1]

        obj1 = np.nan_to_num(f1,nan=100)
        obj2 = np.nan_to_num(f2,nan=100)
        
        
        g1 = obj1-1.5
        g2 = -obj1+0.50
        g3 = np.apply_along_axis(scwb, axis=1,arr=x)

        out["F"] = np.column_stack([obj1, obj2])
        out["G"] = np.column_stack([g1, g2, g3])

problem = MyProblem()
method = get_algorithm("nsga2",
                       pop_size=25,
                       sampling=get_sampling("int_random"),
                       crossover=get_crossover("int_sbx"),
                       mutation=get_mutation("int_pm", eta=3.0),
                       elimate_duplicates=True)

from pymoo.optimize import minimize
import matplotlib.pyplot as plt
from pymoo.util import plotting

res = minimize(problem,
               method,
               termination=('n_gen', 25),
               seed=0,
               save_history=True,
               disp=False,verbose=True)

# print("\nDesign Space")
# plotting.plot(res.X, show=False, no_fill=True)
# plt.ylim(0, 200)
# plt.xlim(0, 200)
# plt.show()

# print("\nObjective Space")
# plotting.plot(res.F, no_fill=True)

print("Best solution found: %s" % res.X)
print("Function value: %s" % res.F)
print("Constraint violation: %s" % res.CV)

C:\Users\ekwon.PARAOFFICE\AppData\Local\Programs\Python\Python39\lib\site-packages\PyNite\FEModel3D.py:925: FutureWarning: `GetNode` will be deleted in a future version of PyNite. Use `Nodes[node_name]` instead.
  warnings.warn('`GetNode` will be deleted in a future version of PyNite. Use `Nodes[node_name]` instead.', FutureWarning)


n_gen |  n_eval |   cv (min)   |   cv (avg)   |  n_nds  |     eps      |  indicator  
    1 |      25 |  0.00000E+00 |  2.16110E+02 |       2 |            - |            -
    2 |      50 |  0.00000E+00 |  1.06002E+01 |       4 |  0.172727273 |        ideal
    3 |      75 |  0.00000E+00 |  0.109563112 |       6 |  0.494830683 |        ideal
    4 |     100 |  0.00000E+00 |  0.031911471 |       7 |  0.012646648 |            f
    5 |     125 |  0.00000E+00 |  0.00000E+00 |      13 |  0.050214127 |        ideal
    6 |     150 |  0.00000E+00 |  0.00000E+00 |      13 |  0.048723898 |        ideal
    7 |     175 |  0.00000E+00 |  0.00000E+00 |      13 |  0.00000E+00 |            f
    8 |     200 |  0.00000E+00 |  0.00000E+00 |      14 |  0.007227545 |            f
    9 |     225 |  0.00000E+00 |  0.00000E+00 |      14 |  0.00000E+00 |            f
   10 |     250 |  0.00000E+00 |  0.00000E+00 |      14 |  0.00000E+00 |            f
   11 |     275 |  0.00000E+00 |  0.00000E+00 |      1

In [129]:
for i in res.X:
    print(col_sections.iloc[i[0]]['AISC_Manual_Label'],bm_sections.iloc[i[1]]['AISC_Manual_Label'])

W21X201 W24X131
W21X248 W24X162
W21X223 W24X131
W21X275 W24X192
W21X275 W24X162
W21X275 W24X176
W21X248 W24X176
W21X275 W24X207
W21X223 W24X117
W21X248 W24X146
W21X248 W24X192
W21X223 W24X162
W21X201 W24X117
W21X223 W24X146
W21X201 W24X146


In [130]:
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
%matplotlib qt
fig = plt.figure()
ax = fig.add_subplot(111)

A1 = res.pop.get("F")
A2 = res.pop.get("X")

a1 = np.hsplit(A1,2)[0]
a2 = np.hsplit(A1,2)[1]

plt.scatter(a1,a2)
count=0
for item in A2:                                       # <--
    ax.annotate('(%s)' % item, (A1[count][0],A1[count][1]), textcoords='data', fontsize=10) # <--
    count+=1
plt.ylim(0,150)
plt.xlim(1.5,2.5)
plt.autoscale(enable=True)

plt.show()

In [135]:
#DEFLECTION FROM RISA for W21x201 Column and W24x146 Beam
delta = 1.516
pct_diff = (delta-edk_risa_np([3,12])[0])/edk_risa_np([3,12])[0]*100
print(f'percentage difference between pynite and risa: {pct_diff}%')

percentage difference between pynite and risa: 1.310531082150185%
